In [134]:
import csv
import pandas as pd
import numpy as np


In [195]:
df=pd.read_csv("BWFakeData.csv")
df.head(20)

,category,annualSpent,ads,aur,upt,numberDept,age,income,category2016
0,0,1803.687775,142.380769,42.948598,1.161083,12.716225,42.946216,187498.217598,1
1,0,1775.592653,139.012529,50.230849,2.985973,7.803634,40.850842,792.480414,5
2,0,1775.246543,141.577621,47.209417,1.957586,7.940131,47.405885,82235.799733,1
3,0,1784.890161,136.759603,46.384693,2.712346,9.863272,44.370667,376163.055133,0
4,0,1767.264194,140.923989,50.287142,3.982783,4.223873,44.995181,27092.991635,1
5,0,1759.763104,136.112705,41.540417,1.654019,0.263488,40.458103,2270.085716,2
6,0,1791.208396,131.662562,51.103815,2.184130,11.484819,48.193413,853.827343,1
7,0,1782.157183,140.113919,50.540508,2.690175,16.734445,48.586045,119542.414366,1
8,0,1811.856054,138.929262,56.674919,2.703534,7.724540,45.029880,66916.607749,1
9,0,1836.473122,134.178015,47.631673,1.011591,12.759836,40.603822,422.227217,2


In [239]:
migrationsStacked = df.groupby(['category','category2016']).agg({'annualSpent' : 'sum'})
migrations = migrationsStacked.unstack(level=-1)
migrations.columns= migrations.columns.droplevel()
migrations = migrations.fillna(0)
migrations = migrations.apply(lambda x: 100*x/float(x.sum()),axis=1).round(2)
migrations

category2016,0,1,2,3,4,5
category,,,,,,
0,11.94,19.94,38.16,19.93,6.07,3.95
1,7.59,21.60,13.68,24.29,16.33,16.51
2,6.92,17.24,22.38,21.38,16.56,15.51
3,8.06,19.75,17.63,21.86,18.74,13.96
4,11.29,17.65,20.68,21.93,13.77,14.68


In [243]:

catAnnualSpent = df.groupby('category').sum()['annualSpent']
catAnnualSpent

category
0     89436.038109
1    101776.553982
2     95725.153857
3     28518.968723
4      4566.438072
Name: annualSpent, dtype: float64

In [247]:

catAnnualSpent2 = (catAnnualSpent.values).dot(migrations.values/100)
catAnnualSpent3 = (catAnnualSpent2[0:-1,]).dot(migrations.values/100)

print catAnnualSpent
print catAnnualSpent2
print catAnnualSpent3


category
0     89436.038109
1    101776.553982
2     95725.153857
3     28518.968723
4      4566.438072
Name: annualSpent, dtype: float64
[ 27841.86378173  62758.77082641  75447.34773947  70247.83168391
  43874.21751831  39834.60507356]
[ 23924.04009637  53732.43103642  57552.85240091  61901.42374     43638.512603
  39410.44275197]


In [294]:
# Matrix multiplication
catAnnualSpentEvolution=[]
catAnnualSpentEvolution.append(catAnnualSpent.values)
prevCAS = (catAnnualSpent.values).dot(migrations.values/100)
catAnnualSpentEvolution.append(prevCAS)

for i in range(10):
    prevCAS = prevCAS[0:-1,].dot(migrations.values/100)
    catAnnualSpentEvolution.append( prevCAS)

print pd.DataFrame(catAnnualSpentEvolution)

               0              1             2             3             4  \
0   89436.038109  101776.553982  95725.153857  28518.968723   4566.438072   
1   27841.863782   62758.770826  75447.347739  70247.831684  43874.217518   
2   23924.040096   53732.431036  57552.852401  61901.423740  43638.512603   
3   20833.522116   46226.499116  49298.004046  53226.045577  37366.797574   
4   17916.266423   39745.487790  42417.955997  45750.203129  32097.100535   
5   15402.436312   34171.187864  36470.609751  39333.738426  27595.727851   
6   13242.267241   29378.814805  31355.645263  33817.401791  23725.690143   
7   11385.102406   25258.566493  26958.145112  29074.662814  20398.269563   
8    9788.392522   21716.164483  23177.385050  24997.069942  17537.501985   
9    8415.613808   18670.568594  19926.859503  21491.341581  15077.944981   
10   7235.361242   16052.104039  17132.205713  18477.276109  12963.329979   
11   6220.633873   13800.867542  14729.489739  15885.919970  11145.280364   

In [320]:
catAnnualSpentEvoSum = pd.DataFrame(catAnnualSpentEvolution).sum(axis=1)
catAnnualSpentEvoSum2 = catAnnualSpentEvoSum
newdf = pd.DataFrame({"Default":catAnnualSpentEvoSum,"Alt1":catAnnualSpentEvoSum2,"Year": [i for i in range(12)]})
newdf.to_json(orient="records")

'[{"Alt1":320023.1527425847,"Default":320023.1527425847,"Year":0},{"Alt1":320004.6366233881,"Default":320004.6366233881,"Year":1},{"Alt1":280159.7026286785,"Default":280159.7026286785,"Year":2},{"Alt1":240741.1121874565,"Default":240741.1121874565,"Year":3},{"Alt1":206943.8552753149,"Default":206943.8552753149,"Year":4},{"Alt1":177920.9804516101,"Default":177920.9804516101,"Year":5},{"Alt1":152968.5128987174,"Default":152968.5128987174,"Year":6},{"Alt1":131515.3594516964,"Default":131515.3594516964,"Year":7},{"Alt1":113070.9120632992,"Default":113070.9120632992,"Year":8},{"Alt1":97213.217404951,"Default":97213.217404951,"Year":9},{"Alt1":83579.4942202268,"Default":83579.4942202268,"Year":10},{"Alt1":71857.8403263675,"Default":71857.8403263675,"Year":11}]'

In [335]:
import numpy as np
np.random.rand(11)*2000

array([  969.3421464 ,  1381.65898853,  1890.71798916,   849.05514994,
        1189.8874992 ,  1006.41564519,  1673.14265013,   657.49515612,
        1928.27026313,  1932.17029624,    50.66001185])